# Imports and Constants:

In [1]:
import GEOMODEL as GEOMODEL
import finesse
finesse.configure(plotting=True)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scienceplots
from cycler import cycler
data = np.load('GEO_F_norm_100000_05.npz')
data2 = np.load('GEO_F_rms_100000_05.npz')
data3 = np.load('LIGO_F_rms_100000_05.npz')
data4 = np.load('LIGO_F_norm_100000_05.npz')
plt.style.use(['science','ieee','notebook'])
plt.rcParams.update({'figure.dpi': 150})
plt.rcParams['axes.prop_cycle'] = cycler('color', ['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', '#8C564B', '#E377C2', '#7F7F7F', '#BCBD22', '#17BECF', '#9A0EEA'])
plt.rcParams.update({'font.serif': 'serif', 'font.family' : 'DeJavu Serif'})
plt.rcParams.update({
        'font.size': 21,
        'axes.labelsize': 21,
        'xtick.labelsize': 18,
        'ytick.labelsize': 18,
        'legend.fontsize': 16
    })

FileNotFoundError: [Errno 2] No such file or directory: 'GEO_F_norm_100000_05.npz'

# aLIGO Model:

In [ ]:
kat = finesse.Model()
kat.parse(
    """
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 3995
    var Mtm  40
    var itmT 0.014
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 125
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T=0.03 L=37.5u phi=90
    s prc prm.p2 bs.p1 L=53


    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmx.p1 L=lmichx
    m itmx T=itmT L=37.5u phi=90
    s LX itmx.p2 etmx.p1 L=Larm
    m etmx T=5u L=37.5u phi=89.999875

    #pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    #pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmy.p1 L=lmichy
    m itmy T=itmT L=37.5u phi=0
    s LY itmy.p2 etmy.p1 L=Larm
    m etmy T=5u L=37.5u phi=0.000125

    #pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    #pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Output and squeezing
    ###########################################################################
    s src bs.p4 srm.p1 L=50.525
    m srm T=0.2 L=37.5u phi=-90

    # A squeezed source could be injected into the dark port
    # sq sq1 db=0 angle=90
    # s lsqz sq1.p1 srm.p2

    # Differentially modulate the arm lengths
    fsig(1)
    # sgen darmx LX.h
    # sgen darmy LY.h phase=180
    
    # Output the full quantum noise limited sensitivity
    # qnoised NSR_with_RP srm.p2.o nsr=True
    # Output just the shot noise limited sensitivity
    # qshot NSR_without_RP srm.p2.o nsr=True

    # We could also display the quantum noise and the signal
    # separately by uncommenting these two lines.
    # qnoised noise srm.p2.o
    # pd1 signal srm.p2.o f=fsig
    """
)

# Transfer Function:

In [ ]:
def compute_tF(srm_phi=''):
    # Create new adjusted model
    new_model = GEOMODEL.adjust_model(srm_phi=srm_phi, sq_db=0)
    
    # Create Simulation that injects a gravitational wave over a range of frequencies of GEO model to caclulate transfer function
    new_sim = new_model.deepcopy()
    new_sim.parse(
        f"""
        dof strain sy.dofs.h -1 sy2.dofs.h -1 sx.dofs.h +1 sx2.dofs.h +1
        readout_dc trans_sb optical_node=FI.p3.o
        frequency_response(logspace(0,5,1000), [strain.AC.i], [trans_sb.DC.o])
        """)
    # Run the new Simulation
    new_sim_run = new_sim.run()
    transferFunction = np.abs(new_sim_run['trans_sb.DC.o','strain.AC.i']) #* (data2['F_rms']/data['Fp_norm'])
    
    # Create Simulation that injects a gravitational wave over a range of frequencies of aLIGO model to caclulate transfer function
    new_sim2 = kat.deepcopy()
    new_sim2.srm.phi = srm_phi
    new_sim2.parse(
        f"""
        dof strain2 LX.dofs.h +1 LY.dofs.h -1
        readout_dc trans_sb2 optical_node=srm.p2.o
        frequency_response(logspace(0,5,1000), [strain2.AC.i], [trans_sb2.DC.o])
        """)
    
    # Run the new Simulation
    new_sim_run2 = new_sim2.run()
    transferFunction2 = np.abs(new_sim_run2['trans_sb2.DC.o','strain2.AC.i']) #* (data3['F_rms']/data4['F_norm'])
    
    # Create Simulation that injects a gravitational wave over a range of frequencies of GEO model to caclulate quantum noise
    new_sim3 = new_model.deepcopy()
    new_sim3.parse(
        f"""
        sgen strain1 sy.h -1
        sgen strain2 sy2.h -1
        sgen strain3 sx.h +1
        sgen strain4 sx2.h +1
        
        # Create noise detectors
        qnoised Quantum_noiseGEO FI.p3.o nsr=false
        
        # Sweep across frequencies
        xaxis(fsig.f, log, 1, 100000, 999)
        """)
    # Run the new Simulation
    new_sim_run3 = new_sim3.run()
    quantum_noiseGEO = np.abs(new_sim_run3['Quantum_noiseGEO'])
    
    # Create Simulation that injects a gravitational wave over a range of frequencies of aLIGO model to caclulate quantum noise
    new_sim4 = kat.deepcopy()
    new_sim4.parse(
        f"""
        sgen strainx LX.h +1
        sgen strainy LY.h -1
        
        # Create noise detectors
        qnoised Quantum_noiseLIGO srm.p2.o nsr=false
        
        # Sweep across frequencies
        xaxis(fsig.f, log, 1, 100000, 999)
        """)
    # Run the new Simulation
    new_sim_run4 = new_sim4.run()
    quantum_noiseLIGO = np.abs(new_sim_run4['Quantum_noiseLIGO'])
    
    # Return abs value of array of new data
    return transferFunction, transferFunction2, quantum_noiseGEO, quantum_noiseLIGO

# Graphing Code:

In [ ]:
def plot_tF(tF_List, tFLabelsList, correction_list, colors, dashes, title):
    # Plot the transfer functions by looping through the input TF list and plotting every array value
    plt.rcParams.update({'font.serif': 'serif', 'font.family' : 'DeJavu Serif'}) 
    x_values = np.logspace(0,5,1000)
    plt.figure(figsize=(10, 8))
    plt.rcParams.update({'font.serif': 'serif', 'font.family' : 'DeJavu Serif'}) 
    for k, tF in enumerate(tF_List):
        plt.plot(x_values, tF*correction_list[k], label=tFLabelsList[k], color=colors[k], ls=dashes[k])
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True, which='major', linestyle='-', linewidth=0.75)
    plt.grid(False, which='minor')  # Disable grid lines for minor ticks
    plt.xlim(1e0,1e5)
    # plt.ylim(1e8,1e15)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (W)')
    plt.legend(frameon=True, ncols=2, loc='lower left', title=f'{title} detuning angles')
    plt.tick_params(which='both',  direction = 'out', top=False, left=True, right=False, bottom=True)
    # plt.savefig(f'{title} Bode Plot.pdf',format='pdf', dpi=300)
    plt.show()
    
    return

# Initialize and Create Bode Plots:

In [ ]:
# Choose MSR detunings:
srm_phi_GEO = [0,15,30,45,60,75,90] #0,15,30,45,60,75,90
srm_phi_LIGO = [1,15,30,45,60,75,90] #1,15,30,45,60,75,90

# Calculate GEO600 transfer function sensitivies
tF_List1 = []
tFLabelsList1 = ['Tuned']
for phi in srm_phi_GEO:
    tF_List1.append(compute_tF(srm_phi=phi)[0])
    tFLabelsList1.append(f'$\phi={phi}$')
tFLabelsList1.pop(1)

# Calculate aLIGO transfer function sensitivies
tF_List2 = []
tFLabelsList2 = []
for phi in srm_phi_LIGO:
    tF_List2.append(compute_tF(srm_phi=phi)[1])
    tFLabelsList2.append(f'$\phi={phi}$')
tFLabelsList2.pop(-1)
tFLabelsList2.append('Tuned')

# Calculate GEO600 quantum noise sensitivies
quantumNoiseList = []
quantumNoiseLabels = []
for phi in srm_phi_GEO:
    quantumNoiseList.append(compute_tF(srm_phi=phi)[2])
    quantumNoiseLabels.append(f'$\phi={phi}$')
    
# Calculate aLIGO quantum noise sensitivies
quantumNoiseList2 = []
quantumNoiseLabels2 = []
for phi in srm_phi_LIGO:
    quantumNoiseList2.append(compute_tF(srm_phi=phi)[3])
    quantumNoiseLabels2.append(f'$\phi={phi}$')

In [ ]:
# Calculate Finesse scaling corrections for each detuning curve for GEO and aLIGO:

# For GEO:
tuned_QN = quantumNoiseList[0][-100]
correction_list = []
for i in range(len(quantumNoiseList)):
    correction_list.append(tuned_QN/quantumNoiseList[i][-100])

# For aLIGO:
tuned_QN2 = quantumNoiseList2[0][-100]
correction_list2 = []
for i in range(len(quantumNoiseList2)):
    correction_list2.append(tuned_QN2/quantumNoiseList2[i][-100])

# Calculate Finesse NSR curves:
GEO_nsr = []
aLIGO_nsr = []
oneList = []
for i in range(len(tF_List1)):
    GEO_nsr.append(quantumNoiseList[i]/tF_List1[i])
for i in range(len(tF_List2)):
    aLIGO_nsr.append(quantumNoiseList2[i]/tF_List2[i])
    oneList.append(1)


# Plot Results:

In [ ]:
# Make colors and dash list:
colors = ['midnightblue', '#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', '#8C564B', '#E377C2', '#7F7F7F', '#BCBD22', '#17BECF', '#9A0EEA', '#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', '#8C564B', '#E377C2', '#7F7F7F', '#BCBD22', '#17BECF', '#9A0EEA']
dash_GEO = ['dashed','solid', 'solid', 'solid', 'solid', 'solid', 'solid']
dash_LIGO = ['solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'dashed', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'dashed']

# Plot Transfer Functions:
plot_tF(tF_List1, tFLabelsList1, correction_list, colors, dash_GEO, 'GEO600')
plot_tF(tF_List2, tFLabelsList2, correction_list2, colors, dash_LIGO, 'aLIGO')
plot_tF(aLIGO_nsr, tFLabelsList2, oneList, colors, dash_LIGO, 'aLIGO')

In [ ]:
for i in range(len(srm_phi_LIGO)):
    plt.loglog(np.logspace(1,5,1000), np.abs(quantumNoiseList2[i]), label="1 deg")